In [1]:
import faiss
import h5py
import torch
from torch.nn.functional import normalize
import numpy as np
import pandas as pd

In [20]:
# hdf_path = "../data/tables/20240621/clamp.h5"
hdf_path = "/media/nova/Datasets/sageev-midi/20250320/pitch-histogram.h5"

with h5py.File(hdf_path, "r") as f:
    # get filenames and embeddings from the file
    filenames = np.array([str(filename[0], "utf-8") for filename in f["filenames"][:]])
    embeddings = np.array(f["pitch-histogram"][:])

    df = pd.DataFrame({"embeddings": [np.array(e, dtype=np.float32) for e in embeddings]}, index=filenames)

    # create dataframe with filenames as index
    # torch.nn.functional.normalize(enc[mask].mean(0).cpu().detach(), p=2, dim=0)
    # df = pd.DataFrame(
    #     # {"embeddings": list(e / np.linalg.norm(e, keepdims=True) for e in embeddings)},
    #     {
    #         "embeddings": list(
    #             normalize(torch.tensor(e), p=2, dim=0) for e in embeddings
    #         )
    #     },
    #     index=filenames,
    #     dtype=np.float32,
    # )

print(df.head())
first_entry = df["embeddings"].iloc[0]
print(f"Type of first entry in df['embeddings']: {type(first_entry)}")
print(f"Dtype of first entry: {first_entry.dtype}")
print(f"Shape of first entry: {first_entry.shape}")

                                                                         embeddings
20231220-080-01_0000-0005_t00s00  [0.0, 0.0, 0.18383147, 0.0, 0.0, 0.0, 0.0, 0.8...
20231220-080-01_0000-0005_t00s01  [0.0, 0.0, 0.18383147, 0.0, 0.0, 0.0, 0.0, 0.8...
20231220-080-01_0000-0005_t00s02  [0.0, 0.0, 0.18383147, 0.0, 0.0, 0.0, 0.0, 0.8...
20231220-080-01_0000-0005_t00s03  [0.0, 0.0, 0.18383147, 0.0, 0.0, 0.0, 0.0, 0.8...
20231220-080-01_0000-0005_t00s04  [0.0, 0.0, 0.18383147, 0.0, 0.0, 0.0, 0.0, 0.8...
Type of first entry in df['embeddings']: <class 'numpy.ndarray'>
Dtype of first entry: float32
Shape of first entry: (12,)


In [ ]:
faiss_index = faiss.IndexFlatIP(768)
faiss_index.add(np.array(df["embeddings"]))
faiss.write_index(
    faiss_index, "/media/nova/Datasets/sageev-midi/20250320/pitch-histogram.faiss"
)

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
# new_df = df.copy()  # avoid modifying the original DataFrame


# def rename_index(index):
#     if index.endswith(".mid"):
#         return f"{index[:-4]}_t00s00"  # remove '.mid' and append '_t00s00'
#     return index  # if not ending with '.mid', keep it as is


# new_df.index = [rename_index(idx) for idx in new_df.index]
# new_df.head()
# new_df.to_hdf("specdiff.h5", key="dataset")

In [3]:
def normalize_vector(vec):
    norm = np.linalg.norm(vec)
    return vec / norm if norm != 0 else vec


df["normed_embeddings"] = df["embeddings"].apply(normalize_vector)
df.head()

,embeddings,normed_embeddings
20240117-064-2b_0044-0052.mid,NaN,NaN
20240117-064-2b_0059-0067.mid,NaN,NaN
20240117-064-2b_0112-0119.mid,NaN,NaN
20240117-064-2b_0142-0149.mid,NaN,NaN
20240117-064-2b_0074-0082.mid,NaN,NaN


In [4]:
index = faiss.IndexFlatIP(768)
embedding_array = np.array(df["normed_embeddings"].tolist(), dtype=np.float32)
index.add(embedding_array)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (11356,) + inhomogeneous part.

In [ ]:
k = 5
distances, indices = index.search(
    np.array(
        [df.loc["20240117-064-2b_0044-0052.mid", "normed_embeddings"]],
        dtype=np.float32,
    ),
    k=k,
)

In [12]:
# Print the most similar documents
for i, index in enumerate(indices[0]):
    distance = distances[0][i]
    print(f"Nearest neighbor {i+1}: {df.iloc[index, :].name}, Distance {distance:.05f}")

Nearest neighbor 1: 20240117-064-2b_0044-0052.mid, Distance 1.00000
Nearest neighbor 2: 20240117-064-2b_0037-0044.mid, Distance 0.97732
Nearest neighbor 3: 20240117-064-2b_0014-0022.mid, Distance 0.96528
Nearest neighbor 4: 20240117-064-2b_0007-0014.mid, Distance 0.96513
Nearest neighbor 5: 20240117-064-2b_0067-0074.mid, Distance 0.95847
